In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os


# Import API key
from api_keys import geoapify_key

In [2]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install citipy

Note: you may need to restart the kernel to use updated packages.


In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
5,5,ushuaia,-54.8000,-68.3000,8.81,81,75,3.09,AR,1666108231
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
8,8,sisimiut,66.9395,-53.6735,2.72,98,100,3.79,GL,1666108235
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236




### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
# %%capture --no-display

# Configure the map plot
gmaps.configure(api_key=geoapify_key)

# Display the map
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
print(fig)



Figure(layout=FigureLayout(height='420px'))


# Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
# Create criteria for the perfect vacation climate
# A max temperature lower than 80 degrees but higher than 65.
# Wind speed less than 10 and no clouds

crit_temp = city_data_df['Max Temp'] < 80
crit_temp = city_data_df['Max Temp'] > 65
crit_windSpeed = city_data_df['Wind Speed'] < 10
crit_cloudiness = city_data_df['Cloudiness'] == 0
final_criteria = crit_temp & crit_windSpeed & crit_cloudiness 

# Drop any rows with null values
city_data_df.dropna(how='any',axis=0)
city_data_df

# Display sample data
# Use boolean indexing to filter the weather_df dataframe
ideal_weather = city_data_df[final_criteria]
ideal_weather = city_data_df.dropna()
ideal_weather = city_data_df.reset_index()
city_data_df.head(10)




,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
5,5,ushuaia,-54.8000,-68.3000,8.81,81,75,3.09,AR,1666108231
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
8,8,sisimiut,66.9395,-53.6735,2.72,98,100,3.79,GL,1666108235
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236


# Step 3: Create a new DataFrame called `hotel_df`

In [22]:
# Create hotel_df with hotel name column
hotel_df = pd.DataFrame(city_data_df, columns=['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Hotel Name'])
hotel_df

hotel_df['Hotel Name'] = hotel_df['Hotel Name'].replace(np.nan, "")
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
1,farsund,NO,58.0948,6.8047,100,
2,new norfolk,AU,-42.7826,147.0587,58,
3,jamestown,US,42.0970,-79.2353,77,
4,lanzhou,CN,36.0564,103.7922,48,
5,ushuaia,AR,-54.8000,-68.3000,81,
6,albany,US,42.6001,-73.9662,74,
7,carnarvon,AU,-24.8667,113.6333,76,
8,sisimiut,GL,66.9395,-53.6735,98,
9,saint-pierre,RE,-21.3393,55.4781,68,


# Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 10000,
        "key": geoapify_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df        


,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,NaN
1,farsund,NO,58.0948,6.8047,100,NaN
2,new norfolk,AU,-42.7826,147.0587,58,NaN
3,jamestown,US,42.0970,-79.2353,77,NaN
4,lanzhou,CN,36.0564,103.7922,48,NaN
...,...,...,...,...,...,...
572,sovetskiy,RU,61.3614,63.5842,96,NaN
573,tatarsk,RU,55.2190,75.9828,89,NaN
574,canchungo,GW,12.0672,-16.0333,61,NaN
575,iquique,CL,-20.2208,-70.1431,68,NaN


In [25]:
# Using the template add the hotel marks to the heatmap
info_box_template = """

Name
{Hotel Name}

City
{City}
Country
{Country}

"""


# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]



In [27]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig


Figure(layout=FigureLayout(height='420px'))